In [ ]:
class CNN(nn.Module):
    def __init__(self,num_classes):
        super().__init__()
        
        self.act = nn.Tanh()
        
        #Feature exrtaction part of captioning
        # input: 28x28x1
        self.c1 = nn.Conv2d(1, 3, kernel_size=3, padding=1) # output: 28x28x3 SAME
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2) # output: 14x14x3 

        self.c2 = nn.Conv2d(3, 6, kernel_size=3, padding=1) # output: 14x14x6 SAME
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2) # output: 7x7x6 
        
        self.c3 = nn.Conv2d(6, 12, kernel_size=3, padding=1) # output: 7x7x12 SAME
        
        self.fc1 = nn.Linear(in_features=7*7*12, out_features=60) 

        #


    
    def forward(self, x):
        #Conv Layer 1
        x = self.c1(x)
        x = self.act(x)
        x = self.pool1(x)
        
        #Conv Layer 2        
        x = self.c2(x)
        x = self.act(x)
        x = self.pool2(x)

        #Conv Layer 3
        x = self.c3(x)
        x = self.act(x)

        #Flattening
        x = torch.flatten(x, 1)

        #FC Layer 1
        x = self.fc1(x)
        x = self.act(x)

        #FC Layer 2
        x = self.fc2(x)       
        
        #Softmax
        out = self.logSoftmax(x)

        return out
    

def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs):
    best_val_loss = float('inf')
    best_model = model

    for epoch in range(num_epochs):
        print(epoch, "ANTON")
        model.train()
        train_loss = 0.0
        for batch_nr, (inputs, labels) in enumerate(train_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss = loss.to(device)
            if (batch_nr%300 == 0):
                print("batch_nr:",batch_nr," loss:",loss.item())
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * inputs.size(0)
        
        train_loss /= len(train_loader.dataset)
        
        writer.add_scalar("loss/train", train_loss, epoch)
    
    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
        
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
    
    val_loss /= len(val_loader.dataset)
    
    writer.add_scalar("loss/val", val_loss, epoch)
    
    
    if (epoch + 1) % 10 == 0:
        print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')
    
    # Save the best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = model
    
    writer.flush()
    return best_model

def test_model(model, test_loader, criterion):
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        test_loss = 0.0
        correct = 0
        total = 0
        all_predictions = []
        all_targets = []

        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            all_predictions.extend(predicted.cpu().numpy())
            all_targets.extend(labels.cpu().numpy())

        test_loss /= len(test_loader.dataset)
        accuracy = 100 * correct / total
        
        # Calculate confusion matrix
        conf_matrix = confusion_matrix(all_targets, all_predictions)

    return test_loss, accuracy, conf_matrix